# This Notebook will develop how to explain an Agent and assess its performance.

***Disclaimer***: This file referenced some files in other directories. In order to have working cross referencing it's recommended to start the notebook server from the root directory (`Grid2Op`) of the package and __not__ in the `getting_started` sub directory:
```bash
cd Grid2Op
jupyter notebook
```

***NB*** For more information about how to use the package, a general help can be built locally (provided that sphinx is installed on the machine) with:
```bash
cd Grid2Op
make html
```
from the top directory of the package (usually `Grid2Op`).

Once build, the help can be access from [here](../documentation/html/index.html)

It is recommended to have a look at the [0_basic_functionalities](0_basic_functionalities.ipynb) notebook before getting into this one.

**Objective**

This notebook will cover the basic of how to "code" an Agent that takes action on the powergrid. Examples will be given of "expert agent" that can take actions based on some fixed rules. More generic type of *Agent*, relying for example on machine learning / deep learning will be covered in the notebook [3_TrainingAnAgent](3_TrainingAnAgent.ipynb).

This notebook will also cover the description of the *Observation* class, usefull to take some actions.

In [1]:
import os
import sys
import grid2op

/home/benjamin/.local/lib/python3.6/site-packages/pandapower/io_utils.py:8: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_series_equal, assert_frame_equal


In [2]:
res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

Impossible to automatically add a menu / table of content to this notebook.
You can download "jyquickhelper" package with: 
"pip install jyquickhelper"


## I) Description of the observations

In this paragraph we will cover the observation class. For more information about it, we recommend to have a look at the official documentation, or [here](../documentation/html/observation.html) or in the [Observations.py](../grid2op/Observations.py) files for more information. Only basic concepts are detailed in this notebook.

### I.A) Getting an observation

An observation can be accessed when calling `env.step()`. The next cell is dedicated to create an environment, and to get an observation instance. We use the default `case14_fromfile` from Grid2Op framework.

In [3]:
env = grid2op.make() #"case14_fromfile")

/home/benjamin/.local/lib/python3.6/site-packages/grid2op/MakeEnv.py:352: UserWarning: Your are using only 2 chronics for this environment. More can be download by running, from a command line:
python -m grid2op.download --name "case14_redisp" --path_save PATH\WHERE\YOU\WANT\TO\DOWNLOAD\DATA
  warnings.warn("Your are using only 2 chronics for this environment. More can be download by running, "


To perform a step, as stated on the short description above, we need an action. More information about them is given in the [2_ActionRepresentation](2_ActionRepresentation.ipynb) notebook. Here we just assume we do nothing.

In [4]:
do_nothing_act = env.helper_action_player({})
obs, reward, done, info = env.step(do_nothing_act)

### I.B) Information present in an Observation

In this notebook we will detail only the "CompleteObservation". `Grid2Op` allows to modeled different kind of observations, for example some with incomplete data, or with noisy data etc. CompletelyObservation gives the full state of the powergrid, without any noise. It's the default observation used.

#### a) some of its attributes

An observation has calendar data (eg the time stamp of the observation):

In [5]:
obs.year, obs.month, obs.day, obs.hour_of_day, obs.minute_of_hour, obs.day_of_week

(2019, 1, 1, 0, 10, 1)

It has some powegrid generic information:

In [6]:
print("Number of generators of the powergrid: {}".format(obs.n_gen))
print("Number of loads of the powergrid: {}".format(obs.n_load))
print("Number of powerline of the powergrid: {}".format(obs.n_line))
print("Number of elements connected to each substations in the powergrid: {}".format(obs.sub_info))
print("Total number of elements: {}".format(obs.dim_topo))

Number of generators of the powergrid: 5
Number of loads of the powergrid: 11
Number of powerline of the powergrid: 20
Number of elements connected to each substations in the powergrid: [3 6 4 6 5 6 3 2 5 3 3 3 4 3]
Total number of elements: 56


It has some information about the generators (each generator can be viewed as a point in a 3 dimensional space)

In [7]:
print("Generators active production: {}".format(obs.prod_p))
print("Generators reactive production: {}".format(obs.prod_q))
print("Generators voltage setpoint : {}".format(obs.prod_v))

Generators active production: [78.5        78.2        24.2         0.         76.43335038]
Generators reactive production: [ 68.82544632 101.2536835  -18.48130499   7.04781916   2.09676292]
Generators voltage setpoint : [142.1 142.1   0.2  12.  142.1]


It has some information the loads (each load is a point in a 3 dimensional space too)

In [8]:
print("Loads active consumption: {}".format(obs.load_p))
print("Loads reactive consumption: {}".format(obs.prod_q))
print("Loads voltage (voltage magnitude of the bus to which it is connected) : {}".format(obs.load_v))

Loads active consumption: [23.8 91.6 14.8 43.5  6.8 12.4 28.9  9.   3.6  5.5 12.1]
Loads reactive consumption: [ 68.82544632 101.2536835  -18.48130499   7.04781916   2.09676292]
Loads voltage (voltage magnitude of the bus to which it is connected) : [142.1        142.1          0.19255956 133.5546623  133.97406149
   0.2          0.20142463   0.1992762    0.19862951   0.19580434
   0.19500454]


In this setting a powerline can be viewed as a point in an 8 dimensional space:
   * active flow
   * reactive flow
   * voltage magnitude
   * current flow
   
from both of its end.

It it is then:

In [9]:
print("Origin active flow: {}".format(obs.p_or))
print("Origin reactive flow: {}".format(obs.q_or))
print("Origin current flow: {}".format(obs.a_or))
print("Origin voltage (voltage magnitude to the bus to which the origin end is connected): {}".format(obs.v_or))
print("Extremity active flow: {}".format(obs.p_ex))
print("Extremity reactive flow: {}".format(obs.q_ex))
print("Extremity current flow: {}".format(obs.a_ex))
print("Extremity voltage (voltage magnitude to the bus to which the origin end is connected): {}".format(obs.v_ex))

Origin active flow: [ 4.24481381e+01  3.39852123e+01  2.99064230e+00  8.06334805e+00
 -6.05131417e+00  1.71992452e+00  7.10173883e+00  2.71666784e+01
  3.88266083e+01  3.08086530e+01  1.34397880e+01 -3.31223550e+01
  9.80043690e+00  7.31346839e+00  1.77709528e+01  2.53492791e+01
  1.46047113e+01  2.30848581e+01  3.74145159e-14  2.53492791e+01]
Origin reactive flow: [-16.24287274  18.33963566  10.71194873  11.48488703   4.20049565
  -0.08629129  -0.21724145  -8.14728023  23.88785737  24.83498364
  32.98203229   3.45327523  -1.37624447   4.10839959   8.83058341
  12.26413609  10.24821629  44.28845057  -6.96032236  17.60379384]
Origin current flow: [  184.66148666   156.90369638 31878.19410017 40222.7664396
 21341.95084869  5077.76221812 21035.96488572   115.23465468
   185.21757697   160.78079521   144.70404203   143.96241692
 28569.01180495 24215.31317375 57284.81228441   121.73502221
    77.12846516   215.22848153   290.64716341  1288.7377647 ]
Origin voltage (voltage magnitude to the 

The last informations about the powerlines is the $\rho$ ratio, *ie.* the ratio between the current flow on each powerlines and the its thermal limits. It can be accessed with:

In [10]:
obs.rho

array([0.47976462, 0.40764776, 0.13920785, 0.17564749, 0.09319747,
       0.33260872, 0.09186127, 0.29938842, 0.48120938, 0.87721359,
       0.37595213, 0.37402533, 0.12475709, 0.10574506, 0.82551211,
       0.31627687, 0.20038563, 0.894688  , 0.08542811, 0.37879067])

It also store information of the topology and the state of the powerline.

In [11]:
obs.timestep_overflow # the number of timestep each of the powerline is in overflow (1 powerline per component)
obs.line_status # the status of each powerline: True connected, False disconnected
obs.topo_vect  # the topology vector the each element (generator, load, each end of a powerline) to which the object
# is connected: 1 = bus 1, 2 = bus 2.

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

 More information about this `topology vector` is given in the documentation [here](../documentation/html/observation.html)

#### b) some of its methods

It can be converted to / from flat numpy vector. This function is usefull for interacting with machine learning or to store it, but probably less human readable. It consists in stacking all the above-mentionned information in a single `numpy.float64` vector.

In [12]:
vector_representation_of_observation = obs.to_vect()
vector_representation_of_observation

array([ 2.01900000e+03,  1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+01,  1.00000000e+00,  7.85000000e+01,  7.82000000e+01,
        2.42000000e+01,  0.00000000e+00,  7.64333504e+01,  6.88254463e+01,
        1.01253683e+02, -1.84813050e+01,  7.04781916e+00,  2.09676292e+00,
        1.42100000e+02,  1.42100000e+02,  2.00000000e-01,  1.20000000e+01,
        1.42100000e+02,  2.38000000e+01,  9.16000000e+01,  1.48000000e+01,
        4.35000000e+01,  6.80000000e+00,  1.24000000e+01,  2.89000000e+01,
        9.00000000e+00,  3.60000000e+00,  5.50000000e+00,  1.21000000e+01,
        1.68000000e+01,  6.36000000e+01,  1.05000000e+01,  3.03000000e+01,
        4.80000000e+00,  8.70000000e+00,  2.07000000e+01,  6.40000000e+00,
        2.50000000e+00,  4.00000000e+00,  8.40000000e+00,  1.42100000e+02,
        1.42100000e+02,  1.92559558e-01,  1.33554662e+02,  1.33974061e+02,
        2.00000000e-01,  2.01424634e-01,  1.99276196e-01,  1.98629508e-01,
        1.95804340e-01,  

An observation can be copied, of course:

In [13]:
obs2 = obs.copy()

Or reset:

In [14]:
obs2.reset()
print(obs2.prod_p)

[nan nan nan nan nan]


Or loaded from a vector:

In [15]:
obs2.from_vect(vector_representation_of_observation)
obs2.prod_p

array([78.5       , 78.2       , 24.2       ,  0.        , 76.43335038])

It is also possible to assess whether two observations are equals or not:

In [16]:
obs == obs2

True

For this type of observation, it is also possible to retrieve the topology as a matrix. The topology matrix can be obtained in two different format.

*Format 1*: the `connectivity matrix` which has as many row / columns as the number of elements in the powergrid (remember an element is either an end of a powerline, or a generator or a load) and that says if 2 elements are connected to one another or not:

$$
\left\{
\begin{aligned}
\text{conn mat}[i,j] = 0 & ~\text{element i and j are NOT connected to the same bus}\\
\text{conn mat}[i,j] = 1 & ~\text{element i and j are connected to the same bus, or i and j are both end of the same powerline}\\
\end{aligned}
\right.
$$

In [17]:
obs.connectivity_matrix()

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

This representation has the advantages to always have the same dimension, regardless of the topology of the powergrid.

*Format 2*: the `bus connectivity matrix` has as many row / columns as the number of active buses of the powergrid. It should be understood as followed:

$$
\left\{
\begin{aligned}
\text{bus conn mat}[i,j] = 0 & ~\text{no powerline connect bus i to bus j}\\
\text{bus conn mat}[i,j] = 1 & ~\text{at least a powerline connectes bus i to bus j (or i == j)}\\
\end{aligned}
\right.
$$

In [18]:
obs.bus_connectivity_matrix()

array([[1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
       [1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.]])

#### c) Simulate

As opposed to most RL problems, in this framework we add the possibility to "simulate" the impact of a possible action on the power grid.

This "simulate" method used some forecasts available (forecasts are made available by the way we loaded the data here, with the class `GridStateFromFileWithForecasts`. For this class, only 1 time step ahead forecasts are provided, but this might be adapted in the future).

Note that this `simulate` function can use a different simulator than the one used by the Environment. Fore more information, we encourage you to read the official documentation or if it has been built locally (recommended) to consult [this page](../documentation/html/observation.html#grid2op.Observation.Observation.simulate).

This function will:

1. apply the forecasted injection on the powergrid
2. run a powerflow with the decidated `simulate` powerflow simulator
3. return:
    1. the anticipated observation (after the action has been taken)
    2. the anticipated reward (of this simulated action)
    3. whether or not there has been an error
    4. some more informations
    
From a user point of view, this is the main difference with the previous [pypownet](https://github.com/MarvinLer/pypownet) framework. This "simulation" used to be performed directly by the environment, thus giving a direct access to the Agent to the Environment, which could break the RL framework (it was not the case in the first edition of the Learning to Run A Power Network as the Environment was fully observable).

In [19]:
do_nothing_act = env.helper_action_player({})
obs_sim, reward_sim, is_done_sim, info_sim = obs.simulate(do_nothing_act)

In [20]:
obs_sim.prod_p

array([78.8       , 77.1       , 24.4       ,  0.        , 77.40249963])

In [21]:
obs.prod_p

array([78.5       , 78.2       , 24.2       ,  0.        , 76.43335038])

## II) Taking actions based on these

In this section we will make our first *Agent* that will act based on these observations.

All *Agent* must derived from the grid2op.Agent class. The main function to code for the Agents is the "act" function (more information on the official documentation or - if the documentation has been built locally- [here](../documentation/html/agent.html) ). 

Basically, the Agent receive a reward and an observation, and suggest a new action. Some different *Agent* are pre-define in the grid2op package. We won't expose them here (for more information see the documantation or the [Agent.py](../grid2op/Agent.py) file), but rather we will make a custom Agent.

This *Agent* will select among:

- do nothing 
- disconnecting the powerline having the higher relative flows
- reconnecting a powerline disconnected
- disconnecting the powerline having the lower relative flows

by using `simulate` on the corresponding actions, and choosing the one that has the highest predicted reward.

Note that this kind of Agent is not particularly smart and is given only as an example.

More information about the creation / manipulation of *Action* will be given in the notebook [2_Action_GridManipulation](2_Action_GridManipulation.ipynb)

In [22]:
from grid2op.Agent import Agent
import numpy as np
import pdb


class MyAgent(Agent):
    def __init__(self, action_space):
        # python required method to code
        Agent.__init__(self, action_space)
        self.do_nothing = self.action_space({})
        self.print_next = False
        
    def act(self, observation, reward, done=False):
        i_max = np.argmax(observation.rho)
        new_status_max = np.zeros(observation.rho.shape)
        new_status_max[i_max] = -1
        act_max = self.action_space({"set_line_status": new_status_max})
        
        i_min = np.argmin(observation.rho)
        new_status_min = np.zeros(observation.rho.shape)
        if observation.rho[i_min] > 0:
            # all powerlines are connected, i try to disconnect this one
            new_status_min[i_min] = -1
            act_min = self.action_space({"set_line_status": new_status_min})
        else:
            # at least one powerline is disconnected, i try to reconnect it
            new_status_min[i_min] = 1
#             act_min = self.action_space({"set_status": new_status_min})
            act_min = self.action_space({"set_line_status": new_status_min,
                                         "set_bus": {"lines_or_id": [(i_min, 1)], "lines_ex_id": [(i_min, 1)]}})
    
        _, reward_sim_dn, *_ = observation.simulate(self.do_nothing)
        _, reward_sim_max, *_ = observation.simulate(act_max)
        _, reward_sim_min, *_ = observation.simulate(act_min)
            
        if reward_sim_dn >= reward_sim_max and reward_sim_dn >= reward_sim_min:
            self.print_next = False
            res = self.do_nothing
        elif reward_sim_max >= reward_sim_min:
            self.print_next = True
            res = act_max
            print(res)
        else:
            self.print_next = True
            res = act_min
            print(res)
        return res

We compare this Agent with the Donothing agent (already coded) on the 3 episode made available with this package. To make the comparison more interesting, it's better to use the L2RPN rewards.

In [23]:
from grid2op.main import main
from grid2op.Agent import DoNothingAgent
from grid2op.Reward import L2RPNReward
from grid2op.ChronicsHandler import GridStateFromFileWithForecasts

max_iter = 10  # to make computation much faster we will only consider 50 time steps instead of 287

res = main(nb_episode=1,
           agent_class=DoNothingAgent,
           path_casefile=grid2op.CASE_14_FILE,
           path_chronics=grid2op.CHRONICS_MLUTIEPISODE,
           names_chronics_to_backend=grid2op.NAMES_CHRONICS_TO_BACKEND,
           gridStateclass_kwargs={"gridvalueClass": GridStateFromFileWithForecasts, "max_iter": max_iter},
           reward_class=L2RPNReward
          )
print("The results for DoNothing agent are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results for DoNothing agent are:
	For chronics located at /home/benjamin/.local/lib/python3.6/site-packages/grid2op/data/test_multi_chronics/1
		 - cumulative reward: 199.998003
		 - number of time steps completed: 10 / 10


In [24]:
res = main(nb_episode=1,
           agent_class=MyAgent,
           path_casefile=grid2op.CASE_14_FILE,
           path_chronics=grid2op.CHRONICS_MLUTIEPISODE,
           names_chronics_to_backend=grid2op.NAMES_CHRONICS_TO_BACKEND,
           gridStateclass_kwargs={"gridvalueClass": GridStateFromFileWithForecasts, "max_iter": max_iter},
           reward_class=L2RPNReward
          )
print("The results for the custom agent are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results for the custom agent are:
	For chronics located at /home/benjamin/.local/lib/python3.6/site-packages/grid2op/data/test_multi_chronics/1
		 - cumulative reward: 199.998003
		 - number of time steps completed: 10 / 10


As we can see, there is no change in the performance for both agent (there would be if we didn't limit the episode length to 10 time steps)

This agent is NOT recommended.

**NB** These scores are given if setting `max_iter=-1` in the previous cells. Here, when `max_iter=10` we don't see any difference in the score (max_iter=10 has been set to make the tests of these notebooks run faster)

In [25]:
from grid2op.Agent import PowerLineSwitch
res = main(nb_episode=1,
           agent_class=PowerLineSwitch,
           path_casefile=grid2op.CASE_14_FILE,
           path_chronics=grid2op.CHRONICS_MLUTIEPISODE,
           names_chronics_to_backend=grid2op.NAMES_CHRONICS_TO_BACKEND,
           gridStateclass_kwargs={"gridvalueClass": GridStateFromFileWithForecasts, "max_iter": max_iter},
           reward_class=L2RPNReward
          )
print("The results for the PowerLineSwitch agent are:")
for chron_name, _, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics located at {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

The results for the PowerLineSwitch agent are:
	For chronics located at /home/benjamin/.local/lib/python3.6/site-packages/grid2op/data/test_multi_chronics/1
		 - cumulative reward: 199.998150
		 - number of time steps completed: 10 / 10


We want however to emphasize that do nothing is NOT the best solution, even in this simple case. For example, an agent choosing at each time step to disconnect / reconnect as to greedily maximize the anticipated reward will have a cumulative reward of 199.998***134*** in this situation.

**NB** For these simulations, the score is completely irrealistic. Indeed, no special care has been taken to set the thermal limits to plausible values. This explain the very little different observed between the three agents above.

**NB** These scores are given if setting `max_iter=-1` in the previous cells. Here, when `max_iter=10` we don't see any difference in the score (max_iter=10 has been set to make the tests of these notebooks run faster)